# Assignment 1
## Zelin Li
### The general logic of the pipeline script followed the code template which is given in the assignment description. The exact steps is shown as below:
## 1 Load the data
### Load the data, ensuring two data sources share the same time span (2020-10-01 to 2021-10-01).

In [1]:
## Getting the data

from influxdb import InfluxDBClient # install via "pip install influxdb "
import pandas as pd
import numpy as np

client = InfluxDBClient(host='influxus.itu.dk', port=8086 , username='lsda', password='icanonlyread')
client.switch_database('orkney')

def get_df(results):
    values = results.raw["series"][0]["values"]
    columns = results.raw["series"][0]["columns"]
    df = pd.DataFrame(values, columns = columns).set_index("time")
    df.index = pd.to_datetime(df.index) # Convert to datetime - index
    return df

# Get the power generation data from 2020-10-01 to 2021-10-01 and calculate its average for each 3 hour time period.
generation = client.query("SELECT time,mean(Total) FROM Generation where time > '2020-10-01' and time <= '2021-10-01' GROUP BY time(3h)")

# Get the weather forecasts from 2020-10-01 to 2021-10-01 with the shortest lead time (Lead hours is 1)
wind = client.query("SELECT time,Direction,Speed FROM MetForecasts where time > '2020-10-01' and time <= '2021-10-01' and Lead_hours = '1'") # earlist time '2019-03-29'

gen_df = get_df(generation)
wind_df = get_df(wind)

## 2 Data cleaning
### Now the `gen_df` and `wind_df` are the only needed data, `gen_df` has 2 dimensions which is `time` and `mean`, while `wind_df` has 3 dimensions which is `time`, `Direction` and `Speed`. The following step is to combine these two Pandas dataframe by joining them (left join using the large one, in this case is `gen_df` larger than `wind_df`).

In [2]:
print('Original gen_df data num:',len(gen_df))
print('Original wind_df data num:',len(wind_df))
if (len(gen_df)>len(wind_df)): df = gen_df.join(wind_df) # left join, gen_df
else: df = wind_df.join(gen_df)
print('Combined(joined) data num:',len(df))

# df = df[df['mean'].notnull()] # drop the column without mean 
# print('Removed data with power generation column is null, remained data num:',len(df))

# df = df[df['Speed'].notnull()]
# print('removed data with wind speed column is null, remained data num:',len(df))

df # show the resulting datafram (which is going to use in the following steps)

Original gen_df data num: 2921
Original wind_df data num: 2779
Combined(joined) data num: 2921


,mean,Direction,Speed
time,,,
2020-10-01 00:00:00+00:00,14.254246,NaN,NaN
2020-10-01 03:00:00+00:00,15.582022,NNW,8.94080
2020-10-01 06:00:00+00:00,15.192839,NNW,9.83488
2020-10-01 09:00:00+00:00,11.969000,NNW,9.83488
2020-10-01 12:00:00+00:00,6.322978,NE,4.91744
...,...,...,...
2021-09-30 12:00:00+00:00,27.414634,SSE,15.19936
2021-09-30 15:00:00+00:00,17.657222,SSE,11.17600
2021-09-30 18:00:00+00:00,20.318838,S,8.04672


### Fill the NaN value in `df`, ensure where `mean`, `Direction` and `Speed` are NaN to be replaced by float number (for linear attributes `mean` and `Speed`) or string (for discrete attribute `Direction`). Different filling strategies are implemented, for linear attributes, it's calculated by the average of the previous value (not NaN) and next first not NaN value; for the discrete attributes, it's replaced by the next not NaN value.

In [3]:
def fillnan(df, col_name):
    num=0
    for i in df[col_name].notnull():
        if i is False:
            j=1
            while pd.isna(df[col_name][num+j]): j+=1 # check whether the next one is nan or not, if it is, skip to its next
            if col_name=='Direction': # for discrete feature(s) (here only the Direction)
                df[col_name][num]=df[col_name][num+j] # set the next not nan value as wind direction of this one
            else: # for all other continuous feature(s)
                if pd.isna(df[col_name][num-1]): df[col_name][num]=df[col_name][num+j] # if there is no previous one, just use the next one as this one value
                else: df[col_name][num]=df[col_name][num-1]-(df[col_name][num-1]-df[col_name][num+j])/(j+1) # calculate the average of the previous value (not null) and next first not nan value to replace this nan
            #print(num,':',df[col_name][num],'\t',num+1,':',df[col_name][num+1])
        num+=1
    return df

# Fill all nan
df=fillnan(df,'mean')
df=fillnan(df,'Direction')
df=fillnan(df,'Speed')
df

<ipython-input-3-23fd70aa4958>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  else: df[col_name][num]=df[col_name][num-1]-(df[col_name][num-1]-df[col_name][num+j])/(j+1) # calculate the average of the previous value (not null) and next first not nan value to replace this nan
<ipython-input-3-23fd70aa4958>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col_name][num]=df[col_name][num+j] # set the next not nan value as wind direction of this one


,mean,Direction,Speed
time,,,
2020-10-01 00:00:00+00:00,14.254246,NNW,6.92912
2020-10-01 03:00:00+00:00,15.582022,NNW,8.94080
2020-10-01 06:00:00+00:00,15.192839,NNW,9.83488
2020-10-01 09:00:00+00:00,11.969000,NNW,9.83488
2020-10-01 12:00:00+00:00,6.322978,NE,4.91744
...,...,...,...
2021-09-30 12:00:00+00:00,27.414634,SSE,15.19936
2021-09-30 15:00:00+00:00,17.657222,SSE,11.17600
2021-09-30 18:00:00+00:00,20.318838,S,8.04672


## 3 Define the pipeline
### The idea of constructing the features is to remain `Speed` as a linear feature (float number), but turning `Direction` into a numeric discrete feature (1, 2, 3, 4, etc.), also the output `mean` (power generation) is considered to be linear.
### To turn `Direction` to numeric, a `transform` function was added to class `WindDirectCoder`, this function can generate encoder for all input features and only select pre-defined discrete feature to encode it (or them) to number. I only pre-defined `Direction` as discrete feature.
### Also, an `Hour` feature which is extracted from `time` is added to the code, but through my testing, show out that it's not a good feature for this task (adding this feature will decrease the performance).

In [4]:
## Preprocess the data

from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.base import BaseEstimator, TransformerMixin

discrete_props = ['Direction'] # demonstrate which column of data is discrete feature (indicating others are linear) 

class HourAdder(BaseEstimator,TransformerMixin):
    def fit(self, X, y):
        return self

    def transform(self, X):
        X["Hour"] = X.index.hour
        return X

class WindDirectCoder():
    def fit(self, X, y):
        return self

    def transform(self, X):
        encoders = {} # a dict storing all encoder of each column
        for i in X.columns: # initialize the encoder for different columns
            encoder = preprocessing.LabelEncoder()
            encoder.fit(X.loc[:, i])
            encoders[i] = encoder
        
        data_num = X.shape[0] # check size
        res = np.zeros(data_num).reshape(-1, 1) # initialize with 0
        for i in X.columns:
            codes = encoders[i].transform(X.loc[:, i]) if i in discrete_props else np.array(X.loc[:, i]) # check whether feature is linear or discrete
            res = np.hstack((res, codes.reshape(-1, 1))) # output
        return pd.DataFrame(res[:, 1:],columns=X.columns)

# class FillNaN():
#     def fit(self, X, y):
#         return self
    
#     def transform(self, X):
#         for i in X.columns: fillnan(X,i)
#         return X

pipeline = Pipeline([
    #("FillNaN", FillNaN()), # fill the row which its column without Speed or Direction
    #("HourAdder", HourAdder()), # commit it because bad performance
    ("WindDirectCoder", WindDirectCoder()), # add this Direction feature is good for the model in general
    #("LinReg", LinearRegression()), # use Linear Regression model
    #("RFReg", RandomForestRegressor()), # use Random Forest model
    ("SVR", SVR(gamma=0.1, C=20.0)), # use Support Vector Machine model, which have the best performance
])

## 4 Fit and train, calculate the MAE
### Split the data into train and test (or validation), fit the pipeline and make prediction.

In [5]:
# Fit the pipeline
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(pd.DataFrame(df,columns=['Direction','Speed']), df['mean'], test_size=0.2, random_state=42)
pipeline.fit(x_train,y_train)
y_hat = pipeline.predict(x_test)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, y_hat)

print("Training MAE:", mae)

Training MAE: 3.8228465894587433


## 5 Forecasting

In [6]:
## Do forecasting
# Get all future forecasts regardless of lead time
forecasts = client.query ("SELECT * FROM MetForecasts where time > now()")
for_df = get_df(forecasts)
# Limit to only the newest source time
newest_source_time = for_df["Source_time"].max()
newest_forecasts = for_df.loc[for_df["Source_time"] == newest_source_time].copy()
# only use the Direction and Speed as input X
newest_forecasts = pd.DataFrame(newest_forecasts,columns=['Direction','Speed'])
# Preprocess the forecasts and do predictions in one fell swoop
pipeline.predict(newest_forecasts)

array([14.78630195, 14.78630195, 10.83520938,  8.56278387,  5.33955047,
        2.71085543,  1.09527213,  2.0434687 ,  2.0434687 ,  1.66910107,
        4.33076241,  5.73216082, 14.11500549, 10.5658752 , 15.97172232,
       27.96971896, 27.96971896, 27.69858608, 27.69858608, 30.22142259,
       31.58822683, 31.58822683, 29.8896061 , 29.8896061 , 32.87374355,
       32.45233088, 31.97865787, 27.05013439, 22.91188927, 16.15540226,
       12.64143017, 13.86405931, 10.5658752 , 10.5658752 , 10.5658752 ])